In [281]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import yfinance as yf
import os
import glob
import regex as re
import csv

In [20]:
%pip install transformers==4.25.1

  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0.dev0
    Uninstalling transformers-4.26.0.dev0:
      Successfully uninstalled transformers-4.26.0.dev0
Note: you may need to restart the kernel to use updated packages.


In [5]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import numpy as np


In [4]:
transformers.__version__

'4.25.1'

using finbert (91% accuracy) instead of Loughran and McDonalds (61% accuracy)
https://towardsdatascience.com/how-nlp-has-evolved-for-financial-sentiment-analysis-fb2990d9b3ed

In [132]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

sentiment_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
sentiment_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Clean Text First

In [124]:
mystr = """
I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a â I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's â and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration.
"""
mystr

"\nI mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a â\x80\x93 I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's â\x80\x93 and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration.\n"

In [222]:
mystr2 = re.sub(r'[^A-Za-z0-9.:,!\' ]', '', mystr) #removes special characters such as the â from above
mystr2

"I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a  I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.  And it's  and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration."

In [126]:
mystr3 = re.sub('\s+', ' ', mystr2) #removes multiple spaces to a single space (i.e. there is a  I --> there is a I)
mystr3

"I mean, I can comment a little bit about it. I mean, the corridor that we did very well in with Cuba and there is a I don't know how else to explain it, but there's a black market currency and a regular currency. And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation. And it's and again, it's not just for us, it's for all of our competitors as well. They are all seeing the same deterioration."

In [133]:
sentiment_nlp = pipeline("text-classification", model=sentiment_finbert, tokenizer=sentiment_tokenizer)
results = sentiment_nlp(mystr3)
results

[{'label': 'Neutral', 'score': 0.9691150188446045}]

tokenise string to individual sentences:

In [136]:
import nltk.data

nltk_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [137]:
mystrtok = (nltk_tokenizer.tokenize(mystr3))
mystrtok

['I mean, I can comment a little bit about it.',
 "I mean, the corridor that we did very well in with Cuba and there is a I don't know how else to explain it, but there's a black market currency and a regular currency.",
 "And people are basically choosing to do business in cash in Cuba because they can buy way more on the black market versus paying for things here, where we have to obviously not do that and that's really the situation.",
 "And it's and again, it's not just for us, it's for all of our competitors as well.",
 'They are all seeing the same deterioration.']

FLS-Classification:

Forward-looking statements (FLS) inform investors of managers’ beliefs and opinions about firm's future events or results. Identifying forward-looking statements from corporate reports can assist investors in financial analysis. FinBERT-FLS is a FinBERT model fine-tuned on 3,500 manually annotated sentences from Management Discussion and Analysis section of annual reports of Russell 3000 firms.

There are 3 classifications of FLS:
Specific foward looking statment  (S-FLS)
Non-specific foward looking statment (NS-FLS)
Not foward looking statment (N-FLS)

We are trying to take the average sentiments of all different classes of FLS statements.
E.g.
4 sentences that are classed as S-FLS, with sentiments [0.6, 0.7, 0.8, 0.9]
so the average sentiment is 0.75
--> Hence, repeat for NS-FLS and NFLS

But first, I need to change the sentiment into a single numerical value
i.e. {'label': 'Negative', 'score': 0.5338950753211975} --> single value

so for 
sentence1 = {'label': 'Negative', 'score': 0.5338950753211975}
sentence2 = {'label': 'Neutral', 'score': 0.999138355255127}
sentence3 = {'label': 'Positive', 'score': 0.9999885559082031}

negative can be mapped to -1
neutral to 0
positive to 1

the sentiment score probability is multiplied by their -1, 0 or 1

so:
sentence1 = -0.5338950753211975
sentence2 = 0
sentence 3 = 0.9999885559082031

------------------------------------------------------------------------------------------------
given 
[{'label': 'Not FLS', 'score': 0.9779328107833862},
 {'label': 'Not FLS', 'score': 0.9796538949012756},
 {'label': 'Specific FLS', 'score': 0.8796855211257935}]

and given
[{'label': 'Negative', 'score': 0.5338950753211975},
 {'label': 'Neutral', 'score': 0.999138355255127},
 {'label': 'Positive', 'score': 0.9999885559082031}]

Not FlS score = (-0.5339+0)/2 = -0.26695
Specific FLS score = 0.99998

In [134]:
# FLS classification
fls_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
fls_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')

In [141]:
nltkstr = "Marcelo Fischer : Just to correct the income from operations, the loss from operations for net2phone was minus $1.8 billion. Of EBITDA itself was about minus $100,000, okay? So we are getting closer to reaching EBITDA profitability and based on our 2023 projections, we do hope to exit 2023 with net2phone being an EBITDA-positive company, even as it continues to grow at rates which are probably higher than the average UCaaS play on basically."

nltkstr= (nltk_tokenizer.tokenize(nltkstr))
nltkstr

['Marcelo Fischer : Just to correct the income from operations, the loss from operations for net2phone was minus $1.8 billion.',
 'Of EBITDA itself was about minus $100,000, okay?',
 'So we are getting closer to reaching EBITDA profitability and based on our 2023 projections, we do hope to exit 2023 with net2phone being an EBITDA-positive company, even as it continues to grow at rates which are probably higher than the average UCaaS play on basically.']

In [163]:
#performs FLS-classifications on the above 3 sentences
fls_nlp = pipeline("text-classification", model=fls_finbert, tokenizer=fls_tokenizer)
fls_results = fls_nlp(nltkstr)
fls_results

[{'label': 'Not FLS', 'score': 0.9779328107833862},
 {'label': 'Not FLS', 'score': 0.9796538949012756},
 {'label': 'Specific FLS', 'score': 0.8796855211257935}]

In [164]:
sent_results = sentiment_nlp(nltkstr)
sent_results

[{'label': 'Negative', 'score': 0.5338950753211975},
 {'label': 'Neutral', 'score': 0.999138355255127},
 {'label': 'Positive', 'score': 0.9999885559082031}]

In [167]:
sent_results[0]

{'label': 'Negative', 'score': 0.5338950753211975}

In [181]:
def map_sentiments(sent_results, i):
    if sent_results[i]['label'] == 'Negative':
        return -1*sent_results[i]['score']
    
    elif sent_results[i]['label'] == 'Neutral':
        return 0
    
    elif sent_results[i]['label'] == 'Positive':
        return sent_results[i]['score']

n_flslist = []
s_flslist = []
ns_flslist = []
for i in range(0,len(fls_results)):
    if fls_results[i]['label'] == 'Not FLS':
        n_flslist.append(map_sentiments(sent_results, i))
    elif fls_results[i]['label'] == 'Specific FLS':
        s_flslist.append(map_sentiments(sent_results, i))
    elif fls_results[i]['label'] == 'Non-specific FLS':
        ns_flslist.append(map_sentiments(sent_results, i))

def average_of_list(mylist):
    try:
        return sum(mylist)/len(mylist)
    except ZeroDivisionError: #if empty list, return 0 (assumes to be neutral)
        return 0


n_fls = average_of_list(n_flslist)
s_fls = average_of_list(s_flslist)
ns_fls = average_of_list(ns_flslist)

print("Sentiments:")
print("Not FLS:", n_fls)
print("Specific FLS:", s_fls)
print("Not-specific FLS:", ns_fls)

Sentiments:
Not FLS: -0.26694753766059875
Specific FLS: 0.9999885559082031
Not-specific FLS: 0


Gather a list of all speaker and analyst names for each stock:

In [253]:
def find_speaker_name(mytranscript, speaker_name_list):
    for sentence in mytranscript:
        # text clean up
        sentence = re.sub(r'[^A-Za-z0-9.,:!\' ]', '', sentence)
        sentence = re.sub('\s+', ' ', sentence) 

        # finds the speaker:
        colon_pos = sentence.find(":")
        speaker_name = sentence[:colon_pos]

        # appends speaker name to speaker_name_list:
        speaker_name_list.append(speaker_name)
    

In [326]:
def find_all_speaker_names(sector, stock):
    speaker_name_list = []
    sector_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[1-9]**[1-9]*[1-4].*')
    sector_files.sort(reverse=True)
    for path in sector_files: # for every path of the stock
        mytranscript = pd.read_csv(path).iloc[[2]].values[0][0] 
        mytranscript = mytranscript.splitlines() # finds transcript
        find_speaker_name(mytranscript, speaker_name_list) # finds all speaker names
    speaker_name_list = list(set(speaker_name_list)) #removes all speaker names duplicates 
    write_path = "sectors/"+sector+"/"+stock+"/"+"speaker names.csv"
    if not os.path.exists(write_path):
        np.savetxt(write_path, speaker_name_list, delimiter =", ", fmt ='% s')

In [329]:
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    print(sector, "sector start")
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        find_all_speaker_names(sector, stock)
    print(sector, "sector complete")



automobiles sector start ................................................................
XPEL start
SUCCESS
THRM start
SUCCESS
APTV start
SUCCESS
XPEV start
SUCCESS
WGO start
SUCCESS
CPS start
SUCCESS
LAZR start
SUCCESS
GT start
SUCCESS
AYRO start
SUCCESS
MPAA start
SUCCESS
DAN start
SUCCESS
TSLA start
SUCCESS
RIDE start
SUCCESS
QS start
SUCCESS
SRI start
SUCCESS
GNTX start
SUCCESS
ALV start
SUCCESS
F start
SUCCESS
CAAS start
SUCCESS
ARVL start
SUCCESS
SUP start
SUCCESS
BWA start
SUCCESS
FOXF start
SUCCESS
REE start
SUCCESS
VC start
SUCCESS
TEN start
SUCCESS
GOEV start
SUCCESS
HOG start
SUCCESS
LEA start
SUCCESS
NIU start
SUCCESS
KNDI start
SUCCESS
RACE start
SUCCESS
SMP start
SUCCESS
NIO start
SUCCESS
GM start
SUCCESS
AXL start
SUCCESS
LI start
SUCCESS
GTX start
SUCCESS
PATK start
SUCCESS
LCID start
SUCCESS
FUV start
SUCCESS
WKHS start
SUCCESS
MOD start
SUCCESS
UFAB start
SUCCESS
FSR start
SUCCESS
SYPR start
SUCCESS
HZN start
SUCCESS
ADNT start
SUCCESS
RIVN start
SUCCESS
XL start
SUC

In [328]:
find_all_speaker_names("telecom","IDT")

SUCCESS


In [317]:
speaker_name_list = list(set(speaker_name_list)) #removes all duplicates 

write_path = "sectors/"+sector+"/"+stock+"/"+"speaker names.csv"

if not os.path.exists(write_path):
    np.savetxt(write_path, speaker_name_list, delimiter =", ", fmt ='% s')

In [323]:
speaker_name_list = list(set(speaker_name_list)) #removes all duplicates 

write_path = "sectors/"+sector+"/"+stock+"/"+"speaker names.csv"

if not os.path.exists(write_path):
    np.savetxt(write_path, speaker_name_list, delimiter =", ", fmt ='% s')

Dividing the transcript:
1. into pre-release
2. into

In [ ]:
gather list of all speaker names for every stock

https://www.spglobal.com/marketintelligence/en/news-insights/blog/analyzing-sentiment-in-quarterly-earnings-calls-q2-2022


https://www.amenityanalytics.com/case-studies/earnings-call-transcript-analysis

whole transcript --> net sentiment

------------------------------------------------------------------------------------------------

Pre release:
Whole pre-release --> net sentiment, word complex

Specific foward looking statment (aggregate to a single string) --> one sentiment, word complex
Non Specific Forward looking statement (aggregate) --> one sentiment, word complex
Not Foward looking statement (aggregate) --> one sentiment, word complex
------------------------------------------------------------------------------------------------
Q&A:
Whole Q&A --> net sentiment, word complex

all question (aggregate)--> sentiment, word complex
all reply (aggregate) --> sentiment, word complex

[if none below, return 0]
Specific foward looking statment --> one sentiment, word complex
Non Specific Forward looking statement --> one sentiment, word complex
Not Foward looking statement --> one sentiment, word complex


In [ ]:
TF IDF value

inspired from https://www.spglobal.com/marketintelligence/en/news-insights/blog/analyzing-sentiment-in-quarterly-earnings-calls-q2-2022

USE DICTIONARY??

able to search for the current quarter sentiment
--------------------------------------------------------------------------------------------------------------------
[on a seperate .csv file in every sector folder]

[PER QUARTER]:

average of net sentiment of each sector in that quarter
average net word complexity of each sector in that quarter

average sentiment of each sector (from forward looking statements)
average sentiment of each sector (from non specific forward looking statements)
average sentiment of each sector (from forward looking statements)

average TF-IDF change (similarity change) in that quarter

number of firms that beat EPS
number of firms that didn't beat EPS

